In [1]:
import numpy as np
import time
from ctypes import *
import matplotlib.pyplot as plt
%matplotlib notebook
from andorsdk.pyandor import Andor

# Loading the Andor SDK and the dll
cam = Andor()
dll = cdll.LoadLibrary("C:\\Program Files\\Andor iXon\\Drivers\\atmcd64d")

In [2]:
# Initializing the camera
messageInitialize = cam.Initialize()
if messageInitialize is not None:
    print(messageInitialize)

Andor: error. Unknown error.


In [ ]:
# Setting the initial settings for the camera
cam.SetShutter(1, 2, 0, 0)
cam.SetReadMode(2)

RandomTrackposition = np.array([165, 198, 211, 244])

cam.SetRandomTracks(2, RandomTrackposition)
cam.GetNumberADChannels()
cam.SetADChannel(1)
cam.SetTriggerMode(0)
cam.GetDetector()
cam.SetHSSpeed(1, 0)
cam.SetVSSpeed(3)

In [ ]:
# Setting up settings for a single scan
cam.SetAcquisitionMode(1)
cam.SetShutter(1, 1, 0, 0)

def getdata():
    dim = 512 * 2
    imagearray = c_int * dim
    cimage = imagearray()

    messageGetData = dll.GetAcquiredData(pointer(cimage), dim)
    if messageGetData is not None:
        print(messageGetData)
    
    return cimage

In [ ]:
ii = 0
plt.figure()
while ii < 50:
    # Taking a single scan
    cam.SetExposureTime(0.05)
    cam.StartAcquisition()
    
    messageGetStatus = cam.GetStatus()
    while messageGetStatus != "DRV_IDLE":
        time.sleep(0.01)
        messageGetStatus = cam.GetStatus()
    
    # Getting data and plotting it
    cimage = getdata()
    image = cimage[:]
    imagenp = np.asarray(image)

    track1 = imagenp[0:511]
    track2 = imagenp[512:1023]
    trackdiff = np.subtract(track1, track2)

    plt.cla()
    plt.plot(track1)
    plt.plot(track2)
    plt.plot(trackdiff)
    plt.show()
    
    ii += 1

In [ ]:
# Shutting down the camera
messageShutdown = cam.ShutDown()
print(messageShutdown)

In [ ]:
dll.ShutDown()